# Mistral

previous model (pm)

1) Setting Up Libraries

run only once

In [6]:
!pip install -q numpy
!pip install -q torch
!pip install -q tqdm
!pip install -q PyPDF2
!pip install -q langchain_text_splitters
!pip install -q sentence_transformers
!pip install -q hnswlib
!pip install -q transformers
!pip install -q pytesseract

In [8]:
# 1.  Standard libraries 
import os
import re
import pickle
import warnings
from typing import List, Dict 

# 2. Third Party libraries 
import numpy as np 
import torch
from tqdm.auto import tqdm
from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import hnswlib
from transformers import AutoModelForCausalLM, AutoTokenizer
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

warnings.filterwarnings('ignore')

print("All libraries correctly installed and imported")

All libraries correctly installed and imported
